<a href="https://colab.research.google.com/github/lailatulbadriyah24/2141720036-machine-learning-2023/blob/main/praktikum-02-tugas-praktikum.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Praktikum 2: Generator Teks dengan RNN**

## **Setup**

### **Import TensorFlow**

In [ ]:
import tensorflow as tf
import numpy as np
import os
import time

### **Download Dataset Shakespeare**

In [ ]:
path_to_file = tf.keras.utils.get_file('shakespeare.txt','https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

### **Load Data**

In [ ]:
# Membaca teks dari file menggunakan mode 'rb' (binary mode) dan mendekode dengan encoding 'utf-8'
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')

# Panjang teks adalah jumlah karakter dalam teks tersebut
print(f'Length of text: {len(text)} characters')

Length of text: 1115394 characters


In [ ]:
# Mencetak 250 karakter pertama dalam teks
print(text[:250])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.



In [ ]:
# Mengidentifikasi karakter-karakter unik dalam teks
vocab = sorted(set(text))

# Mencetak jumlah karakter unik
print(f'{len(vocab)} unique characters')

65 unique characters


## **Olah Teks**

### **Vectorize Teks**

Sebelum training, Anda perlu mengonversi string menjadi representasi numerik. tf.keras.layers.StringLookup dapat mengubah setiap karakter menjadi ID numerik. Caranya adalah teks akan dipecah menjadi token terlebih dahulu.

In [ ]:
# Daftar teks contoh
example_texts = ['abcdefg', 'xyz']

# Memecah teks menjadi karakter-karakter Unicode
chars = tf.strings.unicode_split(example_texts, input_encoding='UTF-8')

# Menampilkan hasil karakter-karakter Unicode
chars

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

sekarang buat tf.keras.layers.StringLookup layer:

In [ ]:
# Membuat lapisan StringLookup untuk mengonversi karakter menjadi ID numerik
ids_from_chars = tf.keras.layers.StringLookup(
    vocabulary=list(vocab),  # Daftar karakter-karakter yang ingin diindeks
    mask_token=None  # Token masking (jika ada), dalam hal ini, tidak ada masking
)

perintah diatas mengconvert token menjadi id

In [ ]:
# Mengonversi karakter-karakter Unicode menjadi ID numerik
ids = ids_from_chars(chars)

# Menampilkan hasil ID numerik
ids

<tf.RaggedTensor [[40, 41, 42, 43, 44, 45, 46], [63, 64, 65]]>

Karena tujuan tutorial ini adalah untuk menghasilkan teks, penting juga untuk membalikkan representasi ini. Untuk ini Anda dapat menggunakan kode
`tf.keras.layers.StringLookup(..., invert=True)`.

In [ ]:
# Membuat lapisan StringLookup untuk mengonversi ID numerik ke karakter-karakter Unicode
chars_from_ids = tf.keras.layers.StringLookup(
    vocabulary=ids_from_chars.get_vocabulary(),  # Menggunakan vocabulary yang telah diindeks sebelumnya
    invert=True,  # Mengatur invert ke True untuk mengonversi kembali dari ID ke karakter
    mask_token=None  # Token masking (jika ada), dalam hal ini, tidak ada masking
)

Lapisan ini mengconvert kembali karakter dari vektor ID, dan mengembalikannya sebagai karakter `tf.RaggedTensor`:

In [ ]:
tf.strings.reduce_join(chars, axis=-1).numpy()

array([b'abcdefg', b'xyz'], dtype=object)

In [ ]:
def text_from_ids(ids):
  return tf.strings.reduce_join(chars_from_ids(ids), axis=-1)

## **Prediksi**

### **Membuat Trianing Set dan Target**

In [ ]:
all_ids = ids_from_chars(tf.strings.unicode_split(text, 'UTF-8'))
all_ids

<tf.Tensor: shape=(1115394,), dtype=int64, numpy=array([19, 48, 57, ..., 46,  9,  1])>

In [ ]:
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)

In [ ]:
for ids in ids_dataset.take(10):
  print(chars_from_ids(ids).numpy().decode('utf-8'))

F
i
r
s
t
 
C
i
t
i


In [ ]:
seq_length = 100

Metode batch memungkinkan Anda dengan mudah mengonversi karakter individual ini menjadi urutan ukuran yang diinginkan.

In [ ]:
sequences = ids_dataset.batch(seq_length+1, drop_remainder=True)

for seq in sequences.take(1):
  print(chars_from_ids(seq))

tf.Tensor(
[b'F' b'i' b'r' b's' b't' b' ' b'C' b'i' b't' b'i' b'z' b'e' b'n' b':'
 b'\n' b'B' b'e' b'f' b'o' b'r' b'e' b' ' b'w' b'e' b' ' b'p' b'r' b'o'
 b'c' b'e' b'e' b'd' b' ' b'a' b'n' b'y' b' ' b'f' b'u' b'r' b't' b'h'
 b'e' b'r' b',' b' ' b'h' b'e' b'a' b'r' b' ' b'm' b'e' b' ' b's' b'p'
 b'e' b'a' b'k' b'.' b'\n' b'\n' b'A' b'l' b'l' b':' b'\n' b'S' b'p' b'e'
 b'a' b'k' b',' b' ' b's' b'p' b'e' b'a' b'k' b'.' b'\n' b'\n' b'F' b'i'
 b'r' b's' b't' b' ' b'C' b'i' b't' b'i' b'z' b'e' b'n' b':' b'\n' b'Y'
 b'o' b'u' b' '], shape=(101,), dtype=string)


akan lebih mudah untuk melihat apa yang dilakukan jika Anda menggabungkan token kembali menjadi string:

In [ ]:
for seq in sequences.take(5):
    print(text_from_ids(seq).numpy())

b'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '
b'are all resolved rather to die than to famish?\n\nAll:\nResolved. resolved.\n\nFirst Citizen:\nFirst, you k'
b"now Caius Marcius is chief enemy to the people.\n\nAll:\nWe know't, we know't.\n\nFirst Citizen:\nLet us ki"
b"ll him, and we'll have corn at our own price.\nIs't a verdict?\n\nAll:\nNo more talking on't; let it be d"
b'one: away, away!\n\nSecond Citizen:\nOne word, good citizens.\n\nFirst Citizen:\nWe are accounted poor citi'


Untuk pelatihan, Anda memerlukan kumpulan data pasangan (input, label). Dimana input dan label merupakan urutan. Pada setiap langkah waktu, inputnya adalah karakter saat ini dan labelnya adalah karakter berikutnya. Berikut adalah fungsi yang mengambil urutan sebagai masukan, menduplikasi, dan menggesernya untuk menyelaraskan masukan dan label untuk setiap langkah waktu:

In [ ]:
def split_input_target(sequence):
  input_text = sequence[:-1]
  target_text = sequence[1:]
  return input_text, target_text

In [ ]:
split_input_target(list("Tensorflow"))

(['T', 'e', 'n', 's', 'o', 'r', 'f', 'l', 'o'],
 ['e', 'n', 's', 'o', 'r', 'f', 'l', 'o', 'w'])

In [ ]:
dataset = sequences.map(split_input_target)

In [ ]:
for input_example, target_example in dataset.take(1):
  print("Input :", text_from_ids(input_example).numpy())
  print("Target:", text_from_ids(target_example).numpy())

Input : b'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou'
Target: b'irst Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '


### **Membuat Batch Training**

Anda menggunakan tf.data untuk membagi teks menjadi sequence yang dapat diatur. Namun sebelum memasukkan data ini ke dalam model, Anda perlu mengacak data dan mengemasnya ke dalam batch.

In [ ]:
# Batch size (ukuran batch) yang digunakan selama pelatihan
BATCH_SIZE = 64

# Buffer size (ukuran buffer) untuk mengacak urutan dataset
# TensorFlow data dirancang untuk bekerja dengan urutan yang mungkin tak terbatas,
# sehingga tidak mencoba untuk mengacak seluruh urutan di dalam memori.
# Sebaliknya, ia mempertahankan buffer di mana ia mengacak elemen.
BUFFER_SIZE = 10000

# Mengonfigurasi dataset dengan mengacak urutan, mengatur ukuran batch,
# dan menggunakan prefetch untuk optimalisasi
dataset = (
    dataset
    .shuffle(BUFFER_SIZE)  # Mengacak urutan dataset
    .batch(BATCH_SIZE, drop_remainder=True)  # Mengatur ukuran batch dengan menghapus sisa data yang tidak cukup untuk satu batch
    .prefetch(tf.data.experimental.AUTOTUNE)  # Menggunakan prefetch untuk optimalisasi
)

# Menampilkan dataset yang telah dikonfigurasi
dataset

<_PrefetchDataset element_spec=(TensorSpec(shape=(64, 100), dtype=tf.int64, name=None), TensorSpec(shape=(64, 100), dtype=tf.int64, name=None))>

### **Buat Model**

In [ ]:
# Jumlah kata dalam vocabulary pada lapisan StringLookup
vocab_size = len(ids_from_chars.get_vocabulary())

# Dimensi embedding
embedding_dim = 256

# Jumlah unit RNN (Recurrent Neural Network)
rnn_units = 1024

In [ ]:
# Mendefinisikan kelas model khusus MyModel
class MyModel(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, rnn_units):
    super().__init__(self)

    # Lapisan embedding untuk mengonversi ID numerik menjadi vektor embedding
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)

    # Lapisan GRU (Gated Recurrent Unit) dengan return_sequences dan return_state
    self.gru = tf.keras.layers.GRU(rnn_units,
                                   return_sequences=True,
                                   return_state=True)

    # Lapisan dense (sepenuhnya terhubung) dengan vocab_size output
    self.dense = tf.keras.layers.Dense(vocab_size)

  def call(self, inputs, states=None, return_state=False, training=False):
    x = inputs

    # Menggunakan lapisan embedding
    x = self.embedding(x, training=training)

    if states is None:
      # Mendapatkan initial_state dari lapisan GRU jika states adalah None
      states = self.gru.get_initial_state(x)

    # Melakukan langkah propagasi pada lapisan GRU
    x, states = self.gru(x, initial_state=states, training=training)

    # Melakukan langkah propagasi pada lapisan dense
    x = self.dense(x, training=training)

    if return_state:
      # Mengembalikan output dan states jika return_state adalah True
      return x, states
    else:
      # Mengembalikan hanya output jika return_state adalah False
      return x

In [ ]:
model = MyModel(
    vocab_size=vocab_size,  # Jumlah kata dalam vocabulary
    embedding_dim=embedding_dim,  # Dimensi embedding
    rnn_units=rnn_units  # Jumlah unit dalam lapisan GRU
)

### **Uji Model**

In [ ]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 66) # (batch_size, sequence_length, vocab_size)


In [ ]:
model.summary()

Model: "my_model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     multiple                  16896     
                                                                 
 gru_1 (GRU)                 multiple                  3938304   
                                                                 
 dense_1 (Dense)             multiple                  67650     
                                                                 
Total params: 4022850 (15.35 MB)
Trainable params: 4022850 (15.35 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices= tf.squeeze(sampled_indices, axis=-1).numpy()

In [ ]:
sampled_indices

array([62, 60, 31, 62, 13, 13,  0, 17, 23, 56,  5, 51, 61, 33,  4, 53, 49,
        5, 57, 11, 45, 28, 36, 60, 48, 19, 28, 25,  7, 51, 61, 53,  7, 28,
       55, 37,  0, 27, 31,  6, 28, 51,  4, 29, 56, 19, 40, 20, 23,  4, 63,
       62, 65, 19, 62, 28, 49,  7, 40, 51, 61, 49, 42, 23, 26, 54, 59, 61,
       34, 36, 50, 53,  8, 10, 27,  9, 41,  7, 44, 26, 20, 63, 31, 29,  1,
       35, 58, 33, 29, 60,  8, 49, 31, 17, 23, 37, 40, 18, 13, 36])

Dekode kode berikut untuk melihat teks yang diprediksi oleh model tidak terlatih ini:

In [ ]:
print("Input:\n", text_from_ids(input_example_batch[0]).numpy())
print()
print("Next Char Predictions:\n", text_from_ids(sampled_indices).numpy())

Input:
 b"ful time!\n\nCAPULET:\nDeath, that hath ta'en her hence to make me wail,\nTies up my tongue, and will no"

Next Char Predictions:
 b"wuRw??[UNK]DJq&lvT$nj&r:fOWuiFOL,lvn,OpX[UNK]NR'Ol$PqFaGJ$xwzFwOj,alvjcJMotvUWkn-3N.b,eMGxRP\nVsTPu-jRDJXaE?W"


### **Train Model**

### **Tambahan optimizer dan fungsi loss**

loss function `tf.keras.losses.sparse_categorical_crossentropy` standar berfungsi dalam kasus ini karena diterapkan di seluruh dimensi terakhir prediksi. Karena model Anda mengembalikan logits, Anda perlu mengatur flag `from_logits`.

In [ ]:
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)

In [ ]:
example_batch_mean_loss = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("Mean loss:        ", example_batch_mean_loss)

Prediction shape:  (64, 100, 66)  # (batch_size, sequence_length, vocab_size)
Mean loss:         tf.Tensor(4.19082, shape=(), dtype=float32)


Model yang baru diinisialisasi tidak boleh terlalu yakin dengan dirinya sendiri, semua log keluaran harus memiliki besaran yang sama. Untuk mengonfirmasi hal ini, Anda dapat memeriksa bahwa eksponensial dari loss rata-rata harus kira-kira sama dengan ukuran kosakata. Loss yang jauh lebih tinggi berarti model tersebut yakin akan jawaban yang salah, dan memiliki inisialisasi yang buruk:

In [ ]:
tf.exp(example_batch_mean_loss).numpy()

66.076965

Konfigurasikan prosedur pelatihan menggunakan metode tf.keras.Model.compile. Gunakan tf.keras.optimizers.Adam dengan argumen default dan fungsi loss.

In [ ]:
model.compile(optimizer='adam', loss=loss)

### **Konfigurasi Checkpoints**

Gunakan `tf.keras.callbacks.ModelCheckpoint` untuk memastikan bahwa checkpoint disimpan selama pelatihan:

In [ ]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

### **Lakukan Proses Training**

In [ ]:
EPOCHS = 10

In [ ]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/10
172/172 [==============================] - 604s 4s/step - loss: 2.5689
Epoch 2/10
172/172 [==============================] - 600s 3s/step - loss: 1.9674
Epoch 3/10
172/172 [==============================] - 598s 3s/step - loss: 1.7002
Epoch 4/10
172/172 [==============================] - 599s 3s/step - loss: 1.5465
Epoch 5/10
172/172 [==============================] - 600s 3s/step - loss: 1.4513
Epoch 6/10
168/172 [============================>.] - ETA: 13s - loss: 1.3852

### **Generate Teks**

Berikut ini membuat prediksi satu langkah:

In [ ]:
class OneStep(tf.keras.Model):
  def __init__(self, model, chars_from_ids, ids_from_chars, temperature=1.0):
    super().__init__()
    self.temperature = temperature
    self.model = model
    self.chars_from_ids = chars_from_ids
    self.ids_from_chars = ids_from_chars

    # Create a mask to prevent "[UNK]" from being generated.
    skip_ids = self.ids_from_chars(['[UNK]'])[:, None]
    sparse_mask = tf.SparseTensor(
        # Put a -inf at each bad index.
        values=[-float('inf')]*len(skip_ids),
        indices=skip_ids,
        # Match the shape to the vocabulary
        dense_shape=[len(ids_from_chars.get_vocabulary())])
    self.prediction_mask = tf.sparse.to_dense(sparse_mask)

  @tf.function
  def generate_one_step(self, inputs, states=None):
    # Convert strings to token IDs.
    input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
    input_ids = self.ids_from_chars(input_chars).to_tensor()

    # Run the model.
    # predicted_logits.shape is [batch, char, next_char_logits]
    predicted_logits, states = self.model(inputs=input_ids, states=states,
                                          return_state=True)
    # Only use the last prediction.
    predicted_logits = predicted_logits[:, -1, :]
    predicted_logits = predicted_logits/self.temperature
    # Apply the prediction mask: prevent "[UNK]" from being generated.
    predicted_logits = predicted_logits + self.prediction_mask

    # Sample the output logits to generate token IDs.
    predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
    predicted_ids = tf.squeeze(predicted_ids, axis=-1)

    # Convert from token ids to characters
    predicted_chars = self.chars_from_ids(predicted_ids)

    # Return the characters and model state.
    return predicted_chars, states

In [ ]:
one_step_model = OneStep(model, chars_from_ids, ids_from_chars)

In [ ]:
start = time.time()
states = None
next_char = tf.constant(['ROMEO:'])
result = [next_char]

for n in range(1000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
print('\nRun time:', end - start)

ROMEO:
A fellow--
Five fire, sir! where is Paul'd Clarence, being good a
chatian to have calm'd in patience, thus I play.

OXFORD:
Now, sir, by yours, nay, my horse: give my bath you are.
When teacher you under thyself he, my own pass
And hide upon thy bosomer for us.

Nousant: flictering, she should be warrant a wood
Dook my nurse; for Northumberland, is the Tower:
The gheen out of them and thy letters answer off.
What is the trouble own of her; belike him a
cause.

POLIXENES:
He must say, sir;
But trud upon thy news?
Before I may purchase you, my harbonies in.

QUEEN MARGARET:
Well heard the strenger be my dispostryurn have.

ESCALUS:
Call it we may dream or lose their weeps. They they parting with
I should have forget for tham for't?
If you lip still report of the abents,
And was a little paradies, and my underfares
Of worlding careless fairy and forband
Slance are rud own heabless.
Go like a gubs o'er; make kiss my master ask you as
your grace grace no less smile and stand with tea

In [ ]:
start = time.time()
states = None
next_char = tf.constant(['ROMEO:', 'ROMEO:', 'ROMEO:', 'ROMEO:', 'ROMEO:'])
result = [next_char]

for n in range(1000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result, '\n\n' + '_'*80)
print('\nRun time:', end - start)

tf.Tensor(
[b"ROMEO:\nJoin night, my lord; you were a prison; fellow,\nAnt say, indeed; 'twere so defend the eft,\n\nAUThis man: his name? see the pleasant tale all my dear hand?\nThen be here baldied mife and misersh?\n\nROMEO:\nNo,\nAnd then my lades, arm my brother biggers;\nUnless this true king, it is yours. and\nthey straight intents, sleweth as they are so.\n\nSLY:\nPlease; Come, he's a woman bring my dangerous to me:\nI have for't, being beleand, and moves as my unthat readies,\nThom for the fish, boy in the shadefolding asking,\nOpholour of happily make the matter,\nTranio, paunt, a feast; 'tis a pies of self\nthem: had it them foes.\n\nMISTREPS OF GARTES:\nThey in tading so she farewell:\nWhen he is a bark? Was this? 'Bounds! why, my masters to you;\nSet it your chance procousion.\n\nLADY ANNE:\nThou hast got them fited: friar, why have and mine!\nFarewell, sister hath out Lilion betwill\nBesum hail.\nBut don, the keeper of Norfolk, shake us before.\n\nKING EDWARD IV:\nAway, 

### **Ekspor Model Generator**

In [ ]:
tf.saved_model.save(one_step_model, 'one_step')
one_step_reloaded = tf.saved_model.load('one_step')

In [ ]:
states = None
next_char = tf.constant(['ROMEO:'])
result = [next_char]

for n in range(100):
  next_char, states = one_step_reloaded.generate_one_step(next_char, states=states)
  result.append(next_char)

print(tf.strings.join(result)[0].numpy().decode("utf-8"))

ROMEO:
Thy birth degree there or no more.

VOLUMNIA:
Ay, march; warwick, inclusanate, madam.

ROMEO:
Who b


# **Tugas Praktikum**

Prosedur pelatihan pada praktikum 2 merupakan prosedur sederhana, yang tidak memberi Anda banyak kendali. Model ini menggunakan "teacher-forcing" yang mencegah prediksi buruk diumpankan kembali ke model, sehingga model tidak pernah belajar untuk pulih dari kesalahan. Jadi, setelah Anda melihat cara menjalankan model secara manual, selanjutnya Anda akan mengimplementasikan custom loop pelatihan. Hal ini memberikan titik awal jika, misalnya, Anda ingin menerapkan pembelajaran kurikulum untuk membantu menstabilkan keluaran open-loop model. Bagian terpenting dari loop pelatihan khusus adalah fungsi langkah pelatihan.

Gunakan `tf.GradientTape` untuk men track nilai gradient. Anda dapat mempelajari lebih lanjut tentang pendekatan ini dengan membaca `eager execution guide`.

Prosedurnya adalah:
1. Jalankan Model dan hitung loss dengan `tf.GradientTape`.
2. Hitung update dan terapkan pada model dengan optimizer

In [ ]:
class CustomTraining(MyModel):
 @tf.function
 def train_step(self, inputs):
      inputs, labels = inputs
      with tf.GradientTape() as tape:
        predictions = self(inputs, training=True)
        loss = self.loss(labels, predictions)
      grads = tape.gradient(loss, model.trainable_variables)
      self.optimizer.apply_gradients(zip(grads, model.trainable_variables))

      return {'loss': loss}

Kode diatas menerapkan `train_step` method sesuai dengan  `Keras' train_step conventions`. Ini opsional, tetapi memungkinkan Anda mengubah perilaku langkah pelatihan dan tetap menggunakan keras `Model.compile` and `Model.fit methods`.

In [ ]:
model = CustomTraining(
     vocab_size=len(ids_from_chars.get_vocabulary()),
     embedding_dim=embedding_dim,
     rnn_units=rnn_units)

In [ ]:
model.compile(optimizer = tf.keras.optimizers.Adam(),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True))

In [ ]:
model.fit(dataset, epochs=1)

172/172 [==============================] - 23s 70ms/step - loss: 2.6993


Atau jika ingin lebih mengetahui dalamnya, kita bisa membuat custom training loop sendiri

In [ ]:
EPOCHS = 10

mean = tf.metrics.Mean()

for epoch in range(EPOCHS):
    start = time.time()

    mean.reset_states()
    for (batch_n, (inp, target)) in enumerate(dataset):
      logs = model.train_step([inp, target])
      mean.update_state(logs['loss'])

      if batch_n % 50 == 0:
         template = f"Epoch {epoch+1} Batch {batch_n} Loss {logs['loss']:.4f}"
         print(template)

 # saving (checkpoint) the model every 5 epochs
      if (epoch + 1) % 5 == 0:
         model.save_weights(checkpoint_prefix.format(epoch=epoch))

    print()
    print(f'Epoch {epoch+1} Loss: {mean.result().numpy():.4f}')
    print(f'Time taken for 1 epoch {time.time() - start:.2f} sec')
    print("_"*80)

model.save_weights(checkpoint_prefix.format(epoch=epoch))

Epoch 1 Batch 0 Loss 2.1579
Epoch 1 Batch 50 Loss 2.0524
Epoch 1 Batch 100 Loss 1.9739
Epoch 1 Batch 150 Loss 1.8555

Epoch 1 Loss: 1.9813
Time taken for 1 epoch 14.38 sec
________________________________________________________________________________
Epoch 2 Batch 0 Loss 1.8018
Epoch 2 Batch 50 Loss 1.7510
Epoch 2 Batch 100 Loss 1.6838
Epoch 2 Batch 150 Loss 1.6812

Epoch 2 Loss: 1.7058
Time taken for 1 epoch 12.08 sec
________________________________________________________________________________
Epoch 3 Batch 0 Loss 1.5939
Epoch 3 Batch 50 Loss 1.6095
Epoch 3 Batch 100 Loss 1.5629
Epoch 3 Batch 150 Loss 1.5461

Epoch 3 Loss: 1.5465
Time taken for 1 epoch 12.11 sec
________________________________________________________________________________
Epoch 4 Batch 0 Loss 1.4616
Epoch 4 Batch 50 Loss 1.4688
Epoch 4 Batch 100 Loss 1.4376
Epoch 4 Batch 150 Loss 1.4129

Epoch 4 Loss: 1.4476
Time taken for 1 epoch 12.07 sec
_____________________________________________________________________

**Jalankan kode diatas dan sebutkan perbedaanya dengan praktikum 2?**

Pada praktikum 2, digunakan pendekatan pelatihan yang lebih umum dan sederhana dengan metode `model.fit` yang sudah terintegrasi dengan TensorFlow. Metode ini mengelola sebagian besar aspek pelatihan, termasuk perhitungan loss, perhitungan gradien, dan pembaruan bobot model secara otomatis.

Sementara itu, pada kode tugas prakikum, digunakan pendekatan pelatihan yang lebih spesifik dan kompleks. Dalam pendekatan ini, didefinisikan metode `train_step` dalam model turunan yang mengatur dengan jelas perhitungan loss, perhitungan gradien, dan pembaruan bobot model. Selain itu, digunakan objek `tf.metrics.Mean` untuk menghitung rata-rata loss selama pelatihan. Pendekatan ini memberikan lebih banyak kemampuan untuk mengendalikan dan menyesuaikan pelatihan model, yang sangat berguna untuk tugas-tugas yang membutuhkan penyesuaian yang spesifik.

Jadi secara keseluruhan, perbedaan utama terletak pada pendekatan pelatihan yang digunakan, di mana kode pada tugas praktikum memberikan tingkat kontrol yang lebih tinggi dan lebih banyak opsi kustomisasi dalam proses pelatihan model.